In [5]:
import os
import sys
import urllib.request
import json
import requests
import re
from bs4 import BeautifulSoup
import urllib
from PIL import Image
import uuid
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from konlpy.tag import Hannanum
from konlpy.tag import Kkma
from konlpy.tag import Komoran
import gensim
from datetime import datetime
import pprint
from ultimate_emoji import deEmojify

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

api_url = 'https://6fg8wpedeo.apigw.ntruss.com/custom/v1/18871/e8c4cfbf6bd0343c951de9ffbcb29bc8accf6d042d71277e3f48d85488718ae3/general'
secret_key = 'V211d2FOR2R5Y1RlanVNSkllZEh2emRDT3NWUlZKc3I='
# 자기 이름을 넣어서 돌리도록 하자
my_name = '정원'
# 아래 쪽에 있는 search_key_word 바꿔서 검색어도 넣도록 하자
# search_key_word = "홍대 우동"     #### <- 테스트 하고 싶은 검색어



# 네이버 클라우드 가입해서 자기 id랑 secret 키를 넣도록 하자
# 해당 사이트 참조 https://yunwoong.tistory.com/153
# https://developers.naver.com/docs/serviceapi/search/blog/blog.md
# 블로그 검색이랑 ocr 둘다 서비스 신청해아 함
# 신청하고 아래 변수에 자기 키랑 시크릿으로 변경
# client_id = "wMXnGlU_z7J6RAmvtkw7"  # <- 발급받은 client id 
# client_secret = "QOrDBMkQ6W"        # <- 발급받은 client secret 


In [6]:
#### Blog extraction 1

def get_infer_text(file_path, secret_key, api_url):
    files = [('file', open(file_path, 'rb'))]
    
    request_json = {'images': [{'format': 'png',
                                    'name': 'demo'
                                }],
                        'requestId': str(uuid.uuid4()),
                        'version': 'V2',
                        'timestamp': int(round(time.time() * 1000))
                    }
    
    payload = {'message': json.dumps(request_json).encode('UTF-8')}
    
    headers = {
    'X-OCR-SECRET': secret_key,
    }            
    response = requests.request("POST", api_url, headers=headers, data=payload, files=files)
    result = response.json()
    return result

def find_fake_blog(last_text):
    fake_keyword = ['제공','협찬']
    genuine_keyword = ['NO협찬','NO광고','내돈내산']
    
    result = False
    for text in last_text:
        if text in genuine_keyword:
            return False
        if text in fake_keyword:
            result = True
    return result

def no_space(text):
    result_remove_both_side = re.sub(r"^\s+|\s+$", "", text)
    return result_remove_both_side

def print_last_text(texts, num_words):
    result_text = []
    count_text = 0
    for i in range(-1, -len(texts), -1):
        txt = no_space(texts[i])
        if len(txt) != 1:
            result_text.append(txt)
            count_text += 1
        if count_text == num_words:
            break
    return result_text

def clean_doc(doc):
    doc = re.sub(r'[^\wㄱ-ㅎ가-힣]', ' ', doc)
    return doc

def contain_fake_keyword(last_ten_texts):    
    corpus = [clean_doc(x) for x in last_ten_texts]
    corpus = [i.strip() for i in corpus]
    # han = Hannanum()
    # komoran = Komoran() 
    kkma = Kkma()

    
    """
    ##이모티콘 삭제를 위해서
    emoji_pattern = re.compile("[" 
    u"\U0001F600-\U0001F64F" # emoticons 
    u"\U0001F300-\U0001F5FF" # symbols & pictographs 
    u"\U0001F680-\U0001F6FF" # transport & map symbols 
    u"\U0001F1E0-\U0001F1FF" # flags (iOS) 
    u"\U0001f926-\U0001f937" # maybe emoji
         "]+", flags=re.UNICODE)
    """
    
    corpus = list(filter(None, corpus))
    tokenized_corpus = [
        kkma.morphs(deEmojify(doc)) for doc in corpus
    ]
    lexicon = gensim.corpora.Dictionary(tokenized_corpus)
    fake_keyword = ['제공','협찬', '소정', '원고료', '수수료', '업체', '수수료', '시식', '이벤트', '쿠폰','제공받','서비스']
    true_keyword = ['스탭', '스텝']
    fake_count = 0
    for x in lexicon.items():
        if x[1] in fake_keyword:
            fake_count += 1
        if x[1] in true_keyword:
            fake_count -= 1            
    if fake_count > 1:
        return True
    return False

def get_img_path(index):
    save_img_path = os.getcwd() + "/untitled_img_" + str(index) +".png"
    if os.path.isfile(save_img_path):
        os.remove(save_img_path)
    return save_img_path

def get_sticker_path(index):
    save_sticker_path = os.getcwd() + "/untitled_img_sticker_" + str(index) +".png"
    if os.path.isfile(save_sticker_path):
        os.remove(save_sticker_path)
    return save_sticker_path

def remove_transparency(save_img_path):
    img = Image.open(save_img_path)
    img = img.convert("RGBA")
    datas = img.getdata()
    
    newData = []
    cutOff = 15
    
    for item in datas:
        if item[3] <= cutOff:
            newData.append((255, 255, 255, 255))
            # RGB의 각 요소가 모두 cutOff 이상이면 transparent하게 바꿔줍니다.
        else:
            newData.append(item)
            # 나머지 요소는 변경하지 않습니다.
    
    img.putdata(newData)
    img.save(save_img_path, "PNG") # PNG 포맷으로 저장합니다.        

def contain_fake_image_keyword(img_list, img_sticker_list):
    result_img = None
    result_sticker = None
    last_text = []
    # save_img_path = "/Users/seonmin.kim/Downloads/untitled_img.png"#"asset/untitled_img.png"
    # save_img_path = os.getcwd() + "/untitled_img.png"
    # if os.path.isfile(save_img_path):
    #     os.remove(save_img_path)
        
    # save_sticker_path = "/Users/seonmin.kim/Downloads/untitled_img_sticker.png"#"asset/untitled_img_sticker.png"
    # save_sticker_path = os.getcwd() + "/untitled_img_sticker.png"
    # if os.path.isfile(save_sticker_path):
    #     os.remove(save_sticker_path)            
        
    if img_list:
        for img_index in range(0,len(img_list)):
            path = img_list.pop()
            save_img_path = get_img_path(img_index)
            urllib.request.urlretrieve(path, save_img_path)
            remove_transparency(save_img_path)
            result_img = get_infer_text(save_img_path, secret_key, api_url)

            if result_img['images'][0]['inferResult'] != 'ERROR':
                for field in result_img['images'][0]['fields']:
                    last_text.append(field['inferText'])
                    # print(field['inferText'])
            if img_index == 3: # 마지막 3개이 image만 ocr 판독하여 검증
                break
        
    if img_sticker_list:
        for img_index in range(0,len(img_sticker_list)):
            path = img_sticker_list.pop()     
            save_sticker_path = get_sticker_path(img_index)              
            urllib.request.urlretrieve(path, save_sticker_path)
            result_sticker = get_infer_text(save_sticker_path, secret_key, api_url)
            
            if result_sticker['images'][0]['inferResult'] != 'ERROR':
                for field in result_sticker['images'][0]['fields']:
                    last_text.append(field['inferText'])
                    # print(field['inferText'])
            if img_index == 3: # 마지막 3개이 sticker image만 ocr 판독하여 검증
                break
    
    fake_result = contain_fake_keyword(last_text)
    return fake_result


def get_num_nouns(full_texts):    
    corpus = [clean_doc(x) for x in full_texts]
    # han = Hannanum()
    # komoran = Komoran() 
    kkma = Kkma()
    tokenized_corpus = [
        kkma.nouns(doc) for doc in corpus
    ]
    lexicon = gensim.corpora.Dictionary(tokenized_corpus)
    
    return len(lexicon.items())

def get_num_sentetnces(full_texts):    
    corpus = [clean_doc(x) for x in full_texts]
    # han = Hannanum()
    # komoran = Komoran() 
    kkma = Kkma()
    tokenized_corpus = [
        kkma.sentences(doc) for doc in corpus
    ]
    lexicon = gensim.corpora.Dictionary(tokenized_corpus)
    
    return len(lexicon.items())

def get_num_pos(full_texts):   
     
    corpus = [clean_doc(x) for x in full_texts]
    # han = Hannanum()
    # komoran = Komoran() 
    kkma = Kkma()
    tokenized_corpus = [
        kkma.pos(doc) for doc in corpus
    ]
    lexicon = gensim.corpora.Dictionary(tokenized_corpus)
    print(lexicon)
    return lexicon.items()
            

In [7]:
### Image extraction


# 블로그 방문 함수
from numpy import savez_compressed


def visit_blog_by_chrome(url):
    headers = { 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
               'lang':'ko'}
    # driver = webdriver.Chrome(executable_path=r'/Users/seonmin.kim/Downloads/chromedriver') # 웹드라이버가 설치된 경로를 지정해주시면 됩니다.
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(0.5)
    html = driver.page_source  # 링크를 따와야 하므로 게시글이 많이 불러진 상태에서 html 객체를 생성
    return html


#블로그 이미지 추출 함수
def get_img_list(html):
    try:
        soup = BeautifulSoup(html, 'html.parser')
        soup = BeautifulSoup(soup.decode(False, formatter=None), None)
        imgs = soup.select('.se-image-resource')
        imgs_sticker = soup.select('.se-sticker-image')
        img_list = list()
        img_sticker_list = list()
        if soup.find("div", attrs={"class":"se-main-container"}): # 스마트 에디터로 작성한 경우
            imgs = soup.find("div", attrs={"class":"se-main-container"}).select('.se-image-resource')  # 큰 사진
            imgs_sticker = soup.find("div", attrs={"class":"se-main-container"}).select('.se-sticker-image') # 스티커 이미지
            for img in imgs:
                if img.get('data-lazy-src') is not None:
                    img_list.append(img['data-lazy-src'])  # 큰사진의 경로
                else:
                    img_list.append(img['src'])  # 작은 사진의 경로 경로
            for img in imgs_sticker:
                if img.get('src') is not None:
                    img_sticker_list.append(img['src'])  # 스티커 사진의 경로
                else:
                    print(f"error on sticker image : {img}")
        else:                            
            imgs = soup.select('.se-image-resource')
            imgs_sticker = soup.select('.se-sticker-image')
            for img in imgs:
                if img.get('data-lazy-src') is not None:
                    img_list.append(img['data-lazy-src'])  # 큰사진의 경로
                else:
                    img_list.append(img['src'])  # 작은 사진의 경로
            for img in imgs_sticker:
                if img.get('src') is not None:
                    img_sticker_list.append(img['src'])  # 스티커 사진의 경로
                else:
                    print(f"error on sticker image : {img}")
            
        return img_list, img_sticker_list
    except:
        print(f"err {img}")


 # 저장경로 생성 함수
def mkdir(title):
    # save_dest = '/Users/seonmin.kim/Downloads/'# for window - 'C:/Users/root/Desktop/'  # 저장을 위한 베이스 경로
    save_dest = os.getcwd()
    if os.path.isdir(save_dest + title):  # 베이스경로에 제목으로 하위 폴더 생성
        pass
    else:
        os.makedirs(save_dest + title)
    return save_dest + title  


# 이미지 저장 함수
def save_img(img_list, save_dest): 
    for i in range(0, len(img_list), 1): 
        ext_idx = (img_list[i].find('?type')) #확장자 찾기 위해 type 문자열을 먼저 찾고 
        ext = (img_list[i][ext_idx-3:ext_idx]) #위에서 찾은 인덱스에서 -3하여 해당 파일의 확장자 확인 
        urllib.request.urlretrieve(img_list[i], save_dest+'/'+str(i+1)+'.'+ext) 
        return len(img_list) 
  

In [8]:
### feature 함수들

# 블로그 제목 추출 함수
def get_title(html):
    soup = BeautifulSoup(html, 'html.parser')
    contents = str(soup.select('.se-fs-'))  # 제목 추출을 위한 셀렉터
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')  # 한글만 따오기 위해
    result = hangul.sub('', contents)
    title = result.strip().split('  ')  # 공백으로 구분
    return title[0]  # 0번째 인덱스가 제목 

def get_blog_totalnumber(html): #블로그 전체 게시글 수
    soup = BeautifulSoup(html,'html.parser')
    soup = BeautifulSoup(soup.decode(False, formatter=None), None)
    contents=[]
    texts = soup.find_all('h4', {'class': 'category_title pcol2'})
    for p_span in texts:
        # pdb.set_trace()
        number = p_span.getText().strip().split(' ')[1][:-2]
        contents.append(number)
    return contents

def visit_blog(url):
    headers = { 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
                'lang':'ko'}
    html = requests.get(url, headers=headers).text
    return html

def input_url(input_url):
    base_url = 'https://blog.naver.com/PostView.naver?blogId='  # 크롤링을 위한 베이스 주소
    # input_url = input('블로그 주소를 입력하세요. 형식은 아래와 같습니다\nhttps://blog.naver.com/아이디/게시글번호\n')
    id = input_url.split('/')[3]  # 아이디 추출
    number = input_url.split('/')[4]  # 게시글번호 추출
    url = base_url + id + '&logNo=' + number
    return url


def get_blog_number(html): #해당 블로그가 속한 메뉴의 게시물의 수
    
    soup1 = BeautifulSoup(html,'html.parser')
    soup1 = BeautifulSoup(soup1.decode(False, formatter=None), None)
    food_info = soup1.find("div", {"class": "blog2_series"})
    if food_info:
        link = food_info.find("a", {"class": "pcol2"})
        source_url = link['href'] if link else '' 

    tempUrl = "https://blog.naver.com" + source_url
    htm12 = visit_blog(tempUrl)

    soup2 = BeautifulSoup(htm12,'html.parser')
    soup2 = BeautifulSoup(soup2.decode(False, formatter=None), None)

    contents=[]
    texts = soup2.find_all('h4', {'class': 'category_title pcol2'})
    for p_span in texts:
        # pdb.set_trace()
        temp = p_span.getText().strip().split()    
        number = [ x[:-2] for x in temp if x[-2:] == "개의"]        
        contents.append(number)
    return contents[0]

def feature_sisAutoPlayYN(html):
    soup = BeautifulSoup(html, 'html.parser')
    soup = BeautifulSoup(soup.decode(False, formatter=None), None)
    t = soup.find_all('script')
    #print(t)
    a = re.findall(r'\sisAutoPlayYN\s=\s\w{5}', str(t))
    if 'true' in a:
        a = True
    else:
        a = False
    return a

#블로거 프로필 info
def get_profile_info(profile_html): #블로거 프로필 info
    soup = BeautifulSoup(profile_html,'html.parser')
    soup = BeautifulSoup(soup.decode(False, formatter=None), None)
    name = []
    texts = soup.find_all('table', {'class': 'profile_info'})
    for text in texts:
        name_ = text.getText().strip().split('\n')[1]
        name.append(name_)
    
    return name

def get_profile_info1(profile_html): #블로거 프로필 info
    soup = BeautifulSoup(profile_html,'html.parser')
    soup = BeautifulSoup(soup.decode(False, formatter=None), None)
    gender = []
    texts = soup.find_all('table', {'class': 'profile_info'})
    for text in texts:
        t_gender = text.getText().strip().split('\n')[5]
        gender.append(t_gender)
    
    return gender



def clean_doc(doc):
    doc = re.sub(r'[^\w]ㄱ-ㅎ가-힣',' ',doc)
    return doc


"""
def body_length(texts): #본문길이(공백포함)
    temp = None
    body_length = []
    texts = [clean_doc(x) for x in texts]
    texts = ''.join(texts)
    temp = len(texts)
    body_length.append(temp)
    
    return body_length
"""
def title_length(texts): #제목길이(단어 공백포함)
    try:
        temp = None
        title_length = []
        texts = [clean_doc(x) for x in texts]
        texts = ''.join(texts)
        temp = len(texts)
        title_length.append(temp)
    
        return title_length
    except:
        return 0

def comment_count(html): #블로그 댓글
    characters = "'?!"
    comment_c = []
    soup = BeautifulSoup(html,'html.parser')
    soup = BeautifulSoup(soup.decode(False, formatter=None), None)
    comments = soup.find_all("em", {"id": "commentCount"})
    if comments:
        for c in comments:
            temp = c.getText().strip().split()
            temp = ''.join( x for x in temp if x not in characters)
            comment_c.append(temp)

        return comment_c
    else:
        return None



# 블로그 제목 추출 함수
def get_last_text(html):
    try:
        soup = BeautifulSoup(html, 'html.parser')
        soup = BeautifulSoup(soup.decode(False, formatter=None), None)
        contents = []
        if soup.find("div", attrs={"class":"se-main-container"}): # 스마트 에디터로 작성한 경우
            texts = soup.find("div", attrs={"class":"se-main-container"}).find_all('div', {'class': re.compile('^se-module se-module-tex.*')})
            for p_span in texts:
                for txt in p_span.find_all('span'):
                    contents.append(txt.get_text())
        else:
            texts = soup.find_all('div', {'class': re.compile('^se-module se-module-tex.*')})
            for p_span in texts:
                for txt in p_span.find_all('span'):
                    contents.append(txt.get_text())
        return contents
    except:
        print(f"err text")    

def profile_url(input_url):
    base_url = 'https://blog.naver.com/profile/intro.naver?blogId='  # 크롤링을 위한 베이스 주소
    # input_url = input('블로그 주소를 입력하세요. 형식은 아래와 같습니다\nhttps://blog.naver.com/아이디/게시글번호\n')
    id = input_url.split('/')[3]  # 아이디 추출
    profile_url = base_url + id
    profile_html = visit_blog(profile_url)
    return profile_html   

def text_length(texts): #본문길이(공백포함)
    try:
        temp = None
        body_length = []
        texts = [clean_doc(x) for x in texts]
        texts = ''.join(texts)
        temp = len(texts)
        body_length.append(temp)
    
        return body_length
    except:
        return 0


# 블로그 해시태그 개수 및 내용 추출 함수
def get_num_hashtags(html):
    try:
        soup = BeautifulSoup(html, 'html.parser')
        soup = BeautifulSoup(soup.decode(False, formatter=None), None)
        
        post_footer = soup.find("div", attrs={"class":"post_footer_contents"})
        post_footer = post_footer.select("span", attrs={"class":"ell"})
        blog_hashtags = []
        for item in post_footer:
            parsed_item = item.get_text().strip()
            if parsed_item != '' and parsed_item != '취소\n확인':
                blog_hashtags.append(parsed_item)
        return blog_hashtags
    except:
        return 0
    

# 블로그 공감 개수 추출 함수
def get_num_likes(html):
    try:
        soup = BeautifulSoup(html, 'html.parser')
        soup = BeautifulSoup(soup.decode(False, formatter=None), None)
        
        likeit = soup.find("span", attrs={"class":"u_likeit_list_btn"})
        likeit_count = likeit.find("em", attrs={"class":"u_cnt"}).get_text()
        return likeit_count
    except:
        return 0
    
# 블로그 이웃 개수 추출 함수
def get_num_neibors(html):
    try:
        soup = BeautifulSoup(html, 'html.parser')
        soup = BeautifulSoup(soup.decode(False, formatter=None), None)
        
        num_neibors = soup.find("div", attrs={"id":"blog-stat"})
        num_neibors = num_neibors.find("ul", attrs={"class":"info"})
        num_neibors = num_neibors.select('em')
        blog_stats = []
        for item in num_neibors:
            blog_stats.append(item.get_text().strip())
        return blog_stats   # 블로그 이웃, 글 보내기, 글 스크랩 순으로 list에 추가
    except:
        return 0
    
    
### weekday, time_delta, datetime(post)        
from datetime import timedelta
default_date = datetime(2022, 1, 1, 0, 0, 0)
def post_datetime(html):
    try:
        soup = BeautifulSoup(html, 'html.parser')
        soup = BeautifulSoup(soup.decode(False, formatter=None), None)
        
        if soup.find("span", attrs={"se_publishDate pcol2"}):
            datetime_str = soup.find("span", attrs={"se_publishDate pcol2"}).get_text().replace(" " , "")
            if re.search(r'시간', datetime_str):
                now = datetime.now()
                hour = re.sub(r'[^0-9]', '', datetime_str)
                datetime_str = now - timedelta(hours = int(hour))
                datetime_str = datetime_str.strftime('%Y.%m.%d.%H:%M')
            elif re.search(r'분', datetime_str):
                now = datetime.now()
                minute = re.sub(r'[^0-9]', '', datetime_str)
                datetime_str = now - timedelta(minutes = int(minute))
                datetime_str = datetime_str.strftime('%Y.%m.%d.%H:%M')
                
        elif soup.find("p", attrs={"date fil5 pcol2 _postAddDate"}):
            datetime_str = soup.find("p", attrs={"date fil5 pcol2 _postAddDate"}).get_text().replace(" " , "")
        
        datetime_obj = datetime.strptime(datetime_str, '%Y.%m.%d.%H:%M')
        time_delta = (datetime_obj - default_date).days
        post_weekday = datetime_obj.weekday()
        return time_delta, post_weekday, str(datetime_obj) #, datetime_obj#, post_weekday
    except:
        return 0
        
### 예약하기 외부 링크 버튼 제공 여/부
def get_reserved(html):
    try:
        soup = BeautifulSoup(html, 'html.parser')
        soup = BeautifulSoup(soup.decode(False, formatter=None), None)
        contents = []
        texts = soup.find_all("a", attrs={"class":"se-placesMap-additional-button se-placesMap-button-reservation"})
        if texts:
            return True
        else:
            return False
    except:
        return 0
            


In [9]:
def fake_recognition(fake_result_text, fake_result_text_):
    if fake_result_text_ or fake_result_text:
        return 1 #print(f"해당 블로그는 가짜 리뷰를 작성하였습니다.")
    else:
        return 0 #print(f"해당 블로그는 진짜 리뷰로 판단됩니다.")
    

In [10]:

client_id = "wMXnGlU_z7J6RAmvtkw7"  # <- 발급받은 client id 
client_secret = "QOrDBMkQ6W"        # <- 발급받은 client secret 
search_key_word = "홍대 우동"     #### <- 테스트 하고 싶은 검색어
encText = urllib.parse.quote(search_key_word)   # <- search keyword 
num_display = 100
num_start = 1
url = "https://openapi.naver.com/v1/search/blog?query=" + encText \
        + "&display=" + str(num_display) + "&start=" + str(num_start) # json 결과
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()


if(rescode==200):
    response_body = json.load(response)
else:
    print("Error Code:" + rescode)


url1 = [] # 0
java_feature = [] # 1
contents = [] # 2
profile_html = [] # 3
name = [] # 4
gender = [] # 5
size_text = [] # 6
size_title = [] # 7
com_count = [] # 8
num_neibors = [] #9
num_likes = [] #10
num_hashtags = [] #11
img_list_size = []
img_sticker_size = []
fake_result = []
post_timedelta = []
post_week = []
post_date = []
get_reserved_link = []
output = pd.DataFrame()

### [:10] 이 부분에서 불러오는 개수 설정 가능
for item in response_body['items'][10:20]:

    # url = input_url(item['link'])
    # html = visit_blog(url)
    # title = get_title(html)
    # texts = get_last_text(html)
    # java_feature.append(feature_sisAutoPlayYN(html))
    # contents.append(get_blog_totalnumber(html))
    # profile_html = profile_url(item['link'])
    # name.append(get_profile_info(profile_html))
    # gender.append(get_profile_info1(profile_html))
    # size_text.append(text_length(texts))
    # size_title.append(title_length(title))
    # com_count.append(comment_count(html))


    url = input_url(item['link'])
    html = visit_blog(url)
    title = get_title(html)
    texts = get_last_text(html)
    java_feature = (feature_sisAutoPlayYN(html))
    contents = (get_blog_totalnumber(html))
    profile_html = profile_url(item['link'])
    name = (get_profile_info(profile_html))
    gender = (get_profile_info1(profile_html))
    size_text = (text_length(texts))
    size_title = (title_length(title))
    com_count = (comment_count(html))
    num_neibors = (get_num_neibors(html))
    num_likes = (get_num_likes(html))
    num_hashtags = len(get_num_hashtags(html))
    img_list, img_sticker_list = get_img_list(html)
    img_list_size = len(img_list)
    img_sticker_size = len(img_sticker_list)
    last_ten_texts = print_last_text(texts, 10)
    post_timedelta, post_week, post_date = post_datetime(html)
    get_reserved_link = get_reserved(html)
    fake_result_text_ = contain_fake_keyword(last_ten_texts)
    fake_result_text = contain_fake_image_keyword(img_list, img_sticker_list)
    fake_result = fake_recognition(fake_result_text, fake_result_text_)

    

    new_df = pd.DataFrame({
    "url" : url,
    "java_feature" : java_feature,
    "contents" : contents,
   # "profile_html" : profile_html,
    "name" : name,
    "gender" : gender,
    "size_text" : size_text,
    "size_title" : size_title,
    "com_count" : com_count,
    "num_neibors" : num_neibors,
    "num_likes" : num_likes,
    "num_hashtags" : num_hashtags,
    "img_list_size" : img_list_size,
    "img_sticker_size" : img_sticker_size,
    "post_timedelta" : post_timedelta,
    "post_week" : post_week,
    "post_date" : post_date,
    "get_reserved_link": get_reserved_link,
    "fake_result" : fake_result
        
    })

    output = output.append(new_df, ignore_index = True)

C:\Users\won\AppData\Local\Temp\ipykernel_29956\2263356920.py:111: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(new_df, ignore_index = True)


In [11]:
output.shape

(10, 18)

In [12]:
output['id'] = output['url']
output['input_user'] = my_name
output.to_json('saved_results.json',orient='records')